## Real Time Object Detection using YOLOv4 with OpenCV and Python
### https://morioh.com/p/74bdafb54c91 (Running YOLO on CPU)

In [1]:
import cv2
import numpy as np
import time

## Load the Network

In [2]:
#load YOLO 
#detects an origin framework of a trained model and calls an appropriate function such as readNetFromCaffe, readNetFromTensorflow, readNetFromTorch, or readNetFromDarknet
#net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg") # dnn = Deep Neural Network module
net = cv2.dnn.readNet("custom-yolov4-tiny-detector_best.weights","custom-yolov4-tiny-detector.cfg")

classes = []
with open ("coco_math.names", "r") as f:
    classes =[line.strip() for line in f.readlines()]                      

In [3]:
print(classes)

['math']


In [4]:
layer_names = net.getLayerNames() #Get the name of all layers of the network.
#print(len(layer_names), layer_names)
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(net.getUnconnectedOutLayers()) #Get the index of the output layers.
print(outputlayers) #Get the name of the output layers.

[[66]
 [78]]
['yolo_30', 'yolo_37']


In [5]:
colors = np.random.uniform(0,255,size=(len(classes),3))

In [30]:
import time

# load the the camera, get the starting time and the frame ID
cap=cv2.VideoCapture(0) #0 for 1st webcam
font = cv2.FONT_HERSHEY_PLAIN #typographie
starting_time = time.time()
frame_id = 0

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
# Define the fps to be equal to 10. Also frame size is passed.
output = cv2.VideoWriter('myface.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

# Extract the frame from the camera
while True: 
    ret,frame= cap.read() # ret : value true or false for next frame, frame : get next frame 
    frame_id+=1
    
    height,width,channels = frame.shape

# Perform the detection
    blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) #reduce 416 to 320    

        
    net.setInput(blob) #feeding input with frame
    outs = net.forward(outputlayers) #frame pass the output layers
    #print(outs[1])


    #Showing info on screen/ get confidence score of algorithm in detecting an object in blob
    class_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:       # take the output for the two classes math and lol
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                #onject detected
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                #rectangle co-ordinaters
                x=int(center_x - w/2)
                y=int(center_y - h/2)
                #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                boxes.append([x,y,w,h]) #put all rectangle areas
                confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                class_ids.append(class_id) #name of the object tha was detected

    #NMSBoxes(bboxes, scores, score_threshold, nms_threshold) : select boxes indexes on treshold
    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6) 

    for i in range(len(boxes)):  
        if i in indexes:
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence= confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
            cv2.putText(frame,label+" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)
            
# Calculate the FPS by deviding the elapsed time by the number of the frames and we show it 
    #elapsed_time = time.time() - starting_time
    #fps=frame_id/elapsed_time
    #cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)
    
    cv2.imshow("Image",frame)
    key = cv2.waitKey(1) #wait 1ms the loop will start again and we will process the next frame
    
    #Write the frame into the file 'outpy.avi'
    output.write(frame)
    
    
    if key == 27: #esc key stops the process
        break;
    
cap.release()    
output.release() 
cv2.destroyAllWindows()